# Analyse datasets in Timbuctoo-instances

In [1]:
%%capture
%run func.ipynb

## List properties per database

In [2]:
import pandas as pd

def get_ds_props(url):
    ds_names = []
    cap_locs = []
    desc_hrefs = []
    desc_types = []
    desc_lens = []
    rl_num = []
    cl_num = []
    rl_locs = []
    cl_locs = []
    rl_ats = []
    rl_completeds = []
    resources = []
    #n_stats = []
    
    # source description
    source_desc = get_sitemap(url)
    for c_url in source_desc.rs_urls: 
        desc_url = None
        desc_type = None
        ds_names.append(c_url.loc.split('/')[6])
        cap_locs.append(c_url.loc) # loc of capability list
        desc_url = c_url.describedby_href
        desc_type = c_url.describedby_type
        
        # capability list
        cap_list = get_sitemap(c_url.loc)
        # urls
        rl_count = 0
        cl_count = 0
        rl_url = None
        cl_url = None
        for rs_url in cap_list.rs_urls:
            if rs_url.capability == 'resourcelist':
                rl_count += 1
                rl_url = rs_url.loc
            if rs_url.capability == 'changelist':
                cl_count += 1
                cl_url = rs_url.loc
        rl_num.append(rl_count)
        cl_num.append(cl_count)
        rl_locs.append(rl_url)
        cl_locs.append(cl_url)
        
        # dataset description is in RDF
        if desc_url is None:
            desc_url = cap_list.describedby_href
            desc_type = cap_list.describedby_type
        desc_len = None
        if desc_url:
            desc_graph = get_graph(desc_url, desc_type)
            desc_len = len(desc_graph)
        desc_hrefs.append(desc_url)
        desc_types.append(desc_type)
        desc_lens.append(desc_len)
        
        # resourcelist                    
        res_list = get_sitemap(rl_url)
        rl_ats.append(res_list.at)
        rl_completeds.append(res_list.completed)
        # up till now we only encounter 1 resource-url in a resource list. 
        if len(res_list.rs_urls) != 1:
            print('warning, %s has not expected number of 1 resource, but %d' % (rl_url, en(res_list.rs_urls)))
        resource = res_list.rs_urls[0].loc
        resources.append(resource)
        
        # resource
#         n_statements = None
#         try:
#             graph = get_quads(resource)
#             n_statements = len(graph)
#         except:
#             print('could not load', resource)
#         n_stats.append(n_statements)
        
        
    return pd.DataFrame({'ds_name': ds_names, 'cap_loc': cap_locs, 
                         'desc_href': desc_hrefs, 'desc_type': desc_types, 'desc_len': desc_lens,
                         'n_rl': rl_num, 'n_cl': cl_num, 'rl_loc': rl_locs, 'cl_loc': cl_locs,
                         'rl_at': rl_ats, 'rl_completed': rl_completeds, 'resource': resources}) #, 'n_statements': n_stats})

In [3]:
pd.options.display.max_colwidth = 0
df = get_ds_props(ANANSI_URL)
display(df.shape, df.head())

(11, 12)

,ds_name,cap_loc,desc_href,desc_type,desc_len,n_rl,n_cl,rl_loc,cl_loc,rl_at,rl_completed,resource
0,wwriters_nl,https://data.anansi.clariah.nl/v5/resourcesync/u38d24500551ccff8d2b0c4f84fc947f45934aa26/wwriters_nl/capabilitylist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u38d24500551ccff8d2b0c4f84fc947f45934aa26/wwriters_nl/description.xml,application/rdf+xml,0,1,1,https://data.anansi.clariah.nl/v5/resourcesync/u38d24500551ccff8d2b0c4f84fc947f45934aa26/wwriters_nl/resourcelist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u38d24500551ccff8d2b0c4f84fc947f45934aa26/wwriters_nl/changelist.xml,2019-01-31 08:04:33.830000+00:00,2019-01-31 13:50:31.833000+00:00,https://data.anansi.clariah.nl/v5/resourcesync/u38d24500551ccff8d2b0c4f84fc947f45934aa26/wwriters_nl/dataset.nq
1,gemeentegeschiedenisnl,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenisnl/capabilitylist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenisnl/description.xml,application/rdf+xml,0,1,1,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenisnl/resourcelist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenisnl/changelist.xml,2019-01-31 08:04:34.028000+00:00,2019-01-31 13:50:31.971000+00:00,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenisnl/dataset.nq
2,dwc,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/dwc/capabilitylist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/dwc/description.xml,application/rdf+xml,7,1,1,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/dwc/resourcelist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/dwc/changelist.xml,2019-01-31 08:04:34.273000+00:00,2019-01-31 13:50:32.104000+00:00,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/dwc/dataset.nq
3,nlgis,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/nlgis/capabilitylist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/nlgis/description.xml,application/rdf+xml,0,1,1,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/nlgis/resourcelist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/nlgis/changelist.xml,2019-01-31 08:04:34.986000+00:00,2019-01-31 13:50:32.226000+00:00,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/nlgis/dataset.nq
4,gemeentegeschiedenis,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenis/capabilitylist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenis/description.xml,application/rdf+xml,0,1,1,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenis/resourcelist.xml,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenis/changelist.xml,2019-01-31 08:04:33.928000+00:00,2019-01-31 13:50:32.386000+00:00,https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenis/dataset.nq


In [4]:
df.to_csv('data/anansi_resources.csv')

In [5]:
df = get_ds_props(HUYDAT_URL)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 12 columns):
ds_name         39 non-null object
cap_loc         39 non-null object
desc_href       39 non-null object
desc_type       39 non-null object
desc_len        39 non-null int64
n_rl            39 non-null int64
n_cl            39 non-null int64
rl_loc          39 non-null object
cl_loc          39 non-null object
rl_at           39 non-null datetime64[ns, UTC]
rl_completed    39 non-null datetime64[ns, UTC]
resource        39 non-null object
dtypes: datetime64[ns, UTC](2), int64(3), object(7)
memory usage: 3.7+ KB


In [6]:
df.to_csv('data/huydat_resources.csv')

## Inspecting datasets

In [7]:
from rdflib import Literal
import requests
from IPython.core.display import display, HTML
import urllib
from urllib.parse import urlparse


def fr(x):
    return '{:12,}'.format(x).replace(',', '.')

def isresolvable(url):
    try:
        response = requests.get(url)
        return response.ok
    except:
        return False

class GraphProps(object):
    
    def __init__(self, g):
        self.statements = len(g)
        self.contexts = list(g.contexts())
        self.netlocs_s = dict()
        self.netlocs_p = dict()
        self.netlocs_o = dict()
        self.literals = 0
        self.predicates = dict()
        for s,p,o in g.triples((None,None,None)):
            self.predicates[str(p)] = self.predicates.get(str(p), 0) + 1
            
            nl = urlparse(str(s)).netloc
            self.netlocs_s[nl] = self.netlocs_s.get(nl, 0) + 1
            
            nl = urlparse(str(p)).netloc
            self.netlocs_p[nl] = self.netlocs_p.get(nl, 0) + 1
            
            if isinstance(o, Literal):
                self.literals += 1
            else:
                nl = urlparse(str(o)).netloc
                self.netlocs_o[nl] = self.netlocs_o.get(nl, 0) + 1
    
    def print_props(self):
        print('contexts  :', fr(len(self.contexts)))
        print('statements:', fr(self.statements))
        print('literals  :', fr(self.literals))
        nar = self.literals/self.statements
        print('literal ratio:', '{:7.2f}'.format(nar).replace('.', ','))
        
        print()
        print('subject netlocs:', len(self.netlocs_s), self.netlocs_s)
        print('object netlocs:', len(self.netlocs_o), self.netlocs_o)
        
        print()
        edo = 'example.org'
        edos = self.netlocs_s.get(edo, 0) + self.netlocs_p.get(edo, 0) + self.netlocs_o.get(edo, 0)
        print('example.org', '| s', self.netlocs_s.get(edo, 0), '| p', self.netlocs_p.get(edo, 0), '| o', self.netlocs_o.get(edo, 0), '| total', edos)
        
        print()
        print('predicate netlocs:', len(self.netlocs_p), '|', self.netlocs_p)
        print('predicates:', len(self.predicates))
        for key in sorted(self.predicates):
            
            if urlparse(key).netloc != 'example.org' and isresolvable(key):
                display(HTML("""<a target="_blank" href="{}">{}</a>""".format(key, key) + ': ' + str(self.predicates[key])))
            else:
                print('%s: %s' % (key, self.predicates[key]))

In [8]:
def fingerprint(resource_url):
    display(HTML("""<h3>fingerprint</h3>"""))
    display(HTML("""<a target="_blank" href="{}">{}</a>""".format(resource_url, resource_url)))
    g = get_quads(resource_url)
    gp = GraphProps(g)
    gp.print_props()

In [9]:
# fingerprint('https://repository.huygens.knaw.nl/v5/resourcesync/u519bd710306620fa7c56d541ae7b9f5b7f57a706/test_wwdocument/dataset.nq')

In [10]:
# rurl ='https://data.anansi.clariah.nl/v5/resourcesync/u74ccc032adf8422d7ea92df96cd4783f0543db3b/gemeentegeschiedenisnl/dataset.nq'
# g = get_quads(rurl)

In [11]:
# from rdflib import URIRef

# max = 5
# tel = 0
# for p, o in g.predicate_objects(URIRef('https://gemeentegeschiedenis.nl/def/geometry_type')):
#     tel += 1
#     print(p, o)
#     if tel >= max:
#         break

In [12]:
dfhuydat = pd.read_csv('data/huydat_resources.csv', index_col=0)
for index, row in dfhuydat.iterrows():
    resource_url = row['resource']
    try:
        fingerprint(resource_url)
    except Exception as ex:
        print(type(ex))

contexts  :            1
statements:       11.852
literals  :        4.688
literal ratio:    0,40

subject netlocs: 12 {'id.emplaces.info': 7057, '': 2860, 'http:': 919, 'www.geonames.org': 384, 'sws.geonames.org': 306, 'dbpedia.org': 91, 'en.wikipedia.org': 91, 'www.w3.org': 124, 'de.wikipedia.org': 2, 'ru.wikipedia.org': 16, 'nds.wikipedia.org': 1, 'uk.wikipedia.org': 1}
object netlocs: 12 {'': 2450, 'id.emplaces.info': 2810, 'www.geonames.org': 354, 'http:': 918, 'sws.geonames.org': 306, 'en.wikipedia.org': 91, 'www.w3.org': 124, 'dbpedia.org': 91, 'ru.wikipedia.org': 16, 'nds.wikipedia.org': 1, 'uk.wikipedia.org': 1, 'de.wikipedia.org': 2}

example.org | s 0 | p 0 | o 0 | total 0

predicate netlocs: 4 | {'http:': 8166, 'www.w3.org': 3654, 'id.emplaces.info': 28, 'timbuctoo.huygens.knaw.nl': 4}
predicates: 42
http://http://id.emplaces.info/vocab/alternateName: 2127
http://http://id.emplaces.info/vocab/alternateURI: 204
http://http://id.emplaces.info/vocab/canonicalURI: 102
http://ht

http://timbuctoo.huygens.knaw.nl/static/v5/vocabulary#hasIndexConfig: 3
http://timbuctoo.huygens.knaw.nl/static/v5/vocabulary#hasViewConfig: 1


contexts  :            1
statements:        1.795
literals  :          519
literal ratio:    0,29

subject netlocs: 19 {'': 817, 'emplaces.namespace.example.org': 306, 'sws.geonames.org': 29, 'emplaces.data.example.org': 464, 'www.w3.org': 41, 'www.geonames.org': 98, 'davidrumsey.georeferencer.com': 10, 'retromap.ru': 2, 'www.dokumentyslaska.pl': 6, 'dbpedia.org': 5, 'en.wikipedia.org': 5, 'ru.wikipedia.org': 3, 'xmlns.com': 3, 'data.huygens.knaw.nl': 1, 'ome-lexikon.uni-oldenburg.de': 1, 'dolny-slask.org.pl': 1, 'www.herder-institut.de': 1, 'github.com': 1, 'purl.org': 1}
object netlocs: 19 {'emplaces.namespace.example.org': 269, 'www.w3.org': 41, '': 597, 'emplaces.data.example.org': 228, 'xmlns.com': 3, 'www.geonames.org': 72, 'sws.geonames.org': 29, 'en.wikipedia.org': 5, 'ru.wikipedia.org': 3, 'www.dokumentyslaska.pl': 6, 'davidrumsey.georeferencer.com': 10, 'dbpedia.org': 5, 'data.huygens.knaw.nl': 1, 'retromap.ru': 2, 'www.herder-institut.de': 1, 'dolny-slask.org.pl': 1, 'github

http://timbuctoo.huygens.knaw.nl/static/v5/vocabulary#hasIndexConfig: 1


contexts  :            1
statements:      529.878
literals  :      133.686
literal ratio:    0,25

subject netlocs: 7 {'example.org': 508099, 'timbuctoo.huygens.knaw.nl': 7555, 'sws.geonames.org': 4236, 'viaf.org': 2137, 'www.dwc.knaw.nl': 7832, 'data.huygens.knaw.nl': 18, 'creativecommons.org': 1}
object netlocs: 7 {'example.org': 374422, 'www.dwc.knaw.nl': 7832, 'timbuctoo.huygens.knaw.nl': 7555, 'sws.geonames.org': 4236, 'viaf.org': 2137, 'data.huygens.knaw.nl': 9, 'creativecommons.org': 1}

example.org | s 508099 | p 248695 | o 374422 | total 1131216

predicate netlocs: 7 | {'www.w3.org': 167563, 'example.org': 248695, 'schema.org': 100766, 'timbuctoo.huygens.knaw.nl': 10718, 'www.geonames.org': 2123, 'purl.org': 12, 'xmlns.com': 1}
predicates: 60
http://example.org/datasets/u33707283d426f900d4d33707283d426f900d4d0d/bia/predicate/Country: 2123
http://example.org/datasets/u33707283d426f900d4d33707283d426f900d4d0d/bia/predicate/aboutPerson: 24978
http://example.org/datasets/u33707283

...

http://schema.org/type: 77822


http://schema.org/userId: 17908
http://schema.org/username: 4555
http://schema.org/variationRef: 35828
http://schema.org/variationRefs: 17908
http://schema.org/vreId: 17908
http://timbuctoo.huygens.knaw.nl/static/v5/vocabulary#hasViewConfig: 1


contexts  :            1
statements:       51.010
literals  :       20.754
literal ratio:    0,41

subject netlocs: 4 {'': 20105, 'www.w3.org': 7261, 'resource.huygens.knaw.nl': 23634, 'data.huygens.knaw.nl': 10}
object netlocs: 4 {'www.w3.org': 7261, '': 14544, 'resource.huygens.knaw.nl': 8447, 'data.huygens.knaw.nl': 4}

example.org | s 0 | p 0 | o 0 | total 0

predicate netlocs: 5 | {'www.w3.org': 15708, 'resource.huygens.knaw.nl': 34697, 'schema.org': 599, 'purl.org': 4, 'timbuctoo.huygens.knaw.nl': 2}
predicates: 91


http://timbuctoo.huygens.knaw.nl/static/v5/vocabulary#hasViewConfig: 2


https://resource.huygens.knaw.nl/emigratie/eaantekeningen: 593
https://resource.huygens.knaw.nl/emigratie/eandere_archiefvormers: 348
https://resource.huygens.knaw.nl/emigratie/eandere_archiefvormers_link: 696
https://resource.huygens.knaw.nl/emigratie/eankernaam: 11
https://resource.huygens.knaw.nl/emigratie/eanno: 22
https://resource.huygens.knaw.nl/emigratie/eannotatie: 40
https://resource.huygens.knaw.nl/emigratie/eannotatie_tekst: 11
https://resource.huygens.knaw.nl/emigratie/earchief: 225
https://resource.huygens.knaw.nl/emigratie/earchieven: 348
https://resource.huygens.knaw.nl/emigratie/earchieven_link: 696
https://resource.huygens.knaw.nl/emigratie/ebestemmings_landen: 696
https://resource.huygens.knaw.nl/emigratie/ebiografie: 67
https://resource.huygens.knaw.nl/emigratie/ebiografie_link: 134
https://resource.huygens.knaw.nl/emigratie/edatum_laatste_verandering: 348
https://resource.huygens.knaw.nl/emigratie/edoelgroep: 865
https://resource.huygens.knaw.nl/emigratie/edoelgroep